- source : https://www.kdnuggets.com/2020/02/audio-data-analysis-deep-learning-python-part-2.html

In [7]:
import pandas as pd
import numpy as np
from numpy import argmax
import matplotlib.pyplot as plt


import librosa
import librosa.display
import IPython.display
import random
import os
from PIL import Image
import pathlib
import csv

from sklearn.model_selection import train_test_split
import keras
import warnings
warnings.filterwarnings('ignore')
from keras import layers
from keras.layers import Activation, Dense, Dropout, Conv2D, Flatten, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling1D, AveragePooling2D, Input, Add
from keras.models import Sequential
from keras.optimizers import SGD

import splitfolders
from keras.preprocessing.image import ImageDataGenerator

### split data training-val

In [6]:
splitfolders.ratio('D:\\Script\\Kuliah\\Tesis\\tutorials\\data\\img_data\\',
                   output='D:\\Script\\Kuliah\\Tesis\\tutorials\\data\\data\\',
                   seed=1337,
                   ratio=(0.8, 0.2))

Copying files: 1000 files [00:04, 233.06 files/s]


### img_augmentation

In [8]:
train_datagen = ImageDataGenerator(rescale=1./255, # rescale all pixel values from 0-255, so aftre this step all our pixel values are in range (0,1)
                                    shear_range=0.2, #to apply some random tranfromations
                                    zoom_range=0.2, #to apply zoom
                                    horizontal_flip=True) # image will be flipper horiztest_datagen = ImageDataGenerator(rescale=1./255)

In [10]:
training_set = train_datagen.flow_from_directory('D:\\Script\\Kuliah\\Tesis\\tutorials\\data\\data\\train\\',
                                                target_size=(64, 64),
                                                batch_size=32,
                                                class_mode='categorical',
                                                shuffle = False)
test_set = train_datagen.flow_from_directory('D:\\Script\\Kuliah\\Tesis\\tutorials\\data\\data\\val\\',
                                            target_size=(64, 64),
                                            batch_size=32,
                                            class_mode='categorical',
                                            shuffle = False )

Found 800 images belonging to 10 classes.
Found 200 images belonging to 10 classes.


### modeling

In [15]:
model = Sequential()
input_shape=(64, 64, 3)

#1st hidden layer
model.add(Conv2D(32, (3, 3), strides=(2, 2), input_shape=input_shape))
model.add(AveragePooling2D((2, 2), strides=(2,2)))
model.add(Activation('relu'))

#2nd hidden layer
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(AveragePooling2D((2, 2), strides=(2,2)))
model.add(Activation('relu'))

#3rd hidden layer
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(AveragePooling2D((2, 2), strides=(2,2)))
model.add(Activation('relu'))

#Flatten
model.add(Flatten())
model.add(Dropout(rate=0.5))

#Add fully connected layer.
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(rate=0.5))

#Output layer
model.add(Dense(10))
model.add(Activation('softmax'))

In [16]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 31, 31, 32)        896       
_________________________________________________________________
average_pooling2d_4 (Average (None, 15, 15, 32)        0         
_________________________________________________________________
activation_6 (Activation)    (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 15, 15, 64)        18496     
_________________________________________________________________
average_pooling2d_5 (Average (None, 7, 7, 64)          0         
_________________________________________________________________
activation_7 (Activation)    (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 7, 7, 64)         

In [17]:
epochs = 200
batch_size = 8
learning_rate = 0.01
decay_rate = learning_rate / epochs
momentum = 0.9
sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
model.compile(optimizer=sgd, loss="categorical_crossentropy", metrics=['accuracy'])

In [18]:
model.fit_generator(
        training_set,
        steps_per_epoch=100,
        epochs=10,
        validation_data=test_set,
        validation_steps=200)

Epoch 1/10
100/100 [==============================] - 116s 1s/step - loss: 2.3431 - accuracy: 0.0294 - val_loss: 2.2838 - val_accuracy: 0.1013
Epoch 2/10
100/100 [==============================] - 109s 1s/step - loss: 2.3210 - accuracy: 0.0397 - val_loss: 2.3020 - val_accuracy: 0.0982
Epoch 3/10
100/100 [==============================] - 120s 1s/step - loss: 2.3216 - accuracy: 0.0325 - val_loss: 2.2960 - val_accuracy: 0.1013
Epoch 4/10
100/100 [==============================] - 113s 1s/step - loss: 2.3225 - accuracy: 0.0388 - val_loss: 2.2821 - val_accuracy: 0.1017
Epoch 5/10
100/100 [==============================] - 131s 1s/step - loss: 2.3234 - accuracy: 0.0306 - val_loss: 2.3065 - val_accuracy: 0.0978
Epoch 6/10
100/100 [==============================] - 120s 1s/step - loss: 2.3233 - accuracy: 0.0450 - val_loss: 2.2855 - val_accuracy: 0.1455
Epoch 7/10
100/100 [==============================] - 137s 1s/step - loss: 2.3232 - accuracy: 0.0512 - val_loss: 2.2749 - val_accuracy: 0.1154

In [20]:
#Model Evaluation
model.evaluate_generator(generator=test_set, steps=50)

[2.288435220718384, 0.09776536375284195]

In [21]:
test_set.reset()
pred = model.predict_generator(test_set, steps=50, verbose=1)

50/50 [==============================] - 20s 395ms/step


In [22]:
predicted_class_indices = np.argmax(pred, axis=1)

labels = (training_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
predictions = predictions[:200]
filenames = test_set.filenames

In [25]:
print(len(filenames), len(predictions))

200 200


In [28]:
results = pd.DataFrame(filenames)

In [30]:
results['pred'] = predictions

In [31]:
results

,0,pred
0,blues\blues00013.png,jazz
1,blues\blues00021.png,jazz
2,blues\blues00026.png,jazz
3,blues\blues00039.png,jazz
4,blues\blues00042.png,jazz
...,...,...
195,rock\rock00084.png,jazz
196,rock\rock00088.png,jazz
197,rock\rock00090.png,jazz
198,rock\rock00093.png,jazz
